# Chapter 4. Molecular Hamiltonians

#### In this section, we are going to discuss about applying VQE to find the lowest energy on a given complicating moleculars.

#### This chapter contains several topics

- Born-Oppenheimer approximation
- Fock space
- Fermionic creation and annihilation operators
- Molecular Hamiltonian in the Hatree-Fock orbitals basis
- Basis sets
- Constructing a fermionic Hamiltonian with Qiskit Nature
- Fermion to qubit mappings
- Constructing a qubit Hamiltonian operator with Qiskit Nature

## I. Installation

#### This part informs several python packages required for our tutorial.

In [ ]:
!pip install numpy      # an open source Python library that is used in almost every field of science and engineering
!pip install qiskit     # an open source SDK for working with quantum computers at the level of pulses, circuits, and application modules
!pip install 'qiskit[visualization]'    # this is for qiskit visualization support
!pip install qiskit.utils               # updated qiskit
!pip install qiskit-nature              # a unique platform that bridges the gap between natural sciences and quantum simulations
!pip install pyscf                      # Python-based Simulations of Chemistry Framework, an open source collection of electronic structure modules powered by Python

In [ ]:
#import packages
import numpy as np

#visualization tools
import matplotlib.pyplot as plt

#qiskit environments and Latex representations of complex array
from qiskit.visualization import array_to_latex, plot_bloch_vector, plot_bloch_multivector, plot_state_qsphere, plot_state_city
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit, transpile
from qiskit import execute, Aer
import qiskit.quantum_info as qi
from qiskit.quantum_info import SparsePauliOp   #updated qiskit
from qiskit.extensions import Initialize
#from qiskit.providers.aer import extensions     #import aer snapshot instructions

#qiskit nature libraries
from qiskit import Aer
#from qiskit_nature.drivers import UnitsType, Molecule

#from qiskit_nature.drivers.second_quantization import ElectronicStructureDriverType, ElectronicStructureMoleculeDriver
#from qiskit_nature.problems.second_quantization import ElectronicStructureProblem
#from qiskit_nature.mappers.second_quantization import ParityMapper, JordanWignerMapper, BravyiKitaevMapper
#from qiskit_nature.converters.second_quantization import QubitConverter

# Electronic Structure Problems with v0.5
# https://qiskit.org/documentation/nature/migration/00b_Electronic_structure_with_v0.5.html
from qiskit_nature.second_q.formats.molecule_info import MoleculeInfo
from qiskit_nature.units import DistanceUnit
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.mappers import QubitConverter
from qiskit_nature.second_q.mappers import ParityMapper, JordanWignerMapper, BravyiKitaevMapper
from qiskit_nature.second_q.properties import ParticleNumber
from qiskit_nature.second_q.transformers import ActiveSpaceTransformer, FreezeCoreTransformer
from qiskit_nature.second_q.operators import FermionicOp
# https://qiskit.org/ecosystem/nature/migration/0.6_c_qubit_converter.html#paulisumop-vs-sparsepauliop
from qiskit_nature import settings

#from qiskit_nature.operators.second_quantization import FermionicOp
#from qiskit_nature.circuit.library.initial_states import HartreeFock
#from qiskit_nature.circuit.library.ansatzes import UCCSD

#Qiskit Nature property framework
#from qiskit_nature.properties import Property, GroupedProperty
# https://qiskit.org/documentation/nature/migration/00b_Electronic_structure_with_v0.5.html
from qiskit_nature.second_q.operators import ElectronicIntegrals

#math libraries
import cmath
import math

## II. Born-Oppenheimer Approximation

#### Since the atomic orbital of an elebron in an atom and the molecular orbital of an electron in a molecule are time-independent stationary states, they can be solved by using time-independent Schrödinger equation: $$ \hat{H} |\Psi \rangle = E | \Psi \rangle $$

#### Where $\hat{H}$ is the non-relateve Hamiltonian operator obtained by quantizing the classical energy in Hamiltonian form (first quantization), and it represents the total energy($E$) of all its particles; $N$ electrons and $M$ nuclei.

#### Let's define a system containing molecules on the coordinates. The origin of the coordinate system can be placed anywhere in free space. It is shown in the figure below

![Coordinate System with Molecules](./img/coordinates.png)

#### The Molecular Hamiltonian in the coordinates is: $$ \hat{H} = -\sum_I \frac{\nabla_{R_I}^{2}}{M_I} -\sum_i \frac{\nabla_{r_i}^{2}}{m_e} -\sum_{I} \sum_{i} \frac{Z_I e^2}{|R_I - r_i|} + \sum_i \sum_{j > i} \frac{e^2}{|r_i - r_j|} + \sum_I \sum_{J > I} \frac{Z_I Z_J e^2}{|R_I - R_J|} $$

#### where

- $\nabla_{r_i}^{2}$ and $\nabla_{R_I}^{2}$ are the second derivate operator with respecto to the position coordinates for electrons and nuclei, that is, $\nabla_{R_I}^{2} = \frac{\partial^2}{\partial_{x_I}^{2}} + \frac{\partial^2}{\partial_{y_I}^{2}} + \frac{\partial^2}{\partial_{z_I}^{2}}$ and likewise for the p<sup>th</sup> electrons.
- $r_{ij} = \left|r_i - r_j\right|, r_{iI} = \left|r_i - R_I\right|, R_{IJ} = \left|R_I - R_J\right|$ are the distances between electrons $i$ and $j$, electron $i$ and nucles $I$, and the nuclei $I$ and $J$ determined by the Eulidean norm
- indexes with upper case such as $I, J$ represent nuclei, and indexes with lower case such as $i, j$ represent electrons
- $Z_I$ represents the atomic number of the nuclei

#### For clarity, we list the operators of the Molecular Hamiltonian in the table below:

| **Operator**                                            	| **Description**                                                                                                                                           	|
|---------------------------------------------------------	|-----------------------------------------------------------------------------------------------------------------------------------------------------------	|
| $\sum_{I} \frac{\nabla_{r_i}^{2}}{M_I}$                 	| Nuclear ***Kinetic*** Energy Operator                                                                                                                       	|
| $\sum_i \frac{\nabla_{r_i}^{2}}{m_e}$                  	| Electron ***Kinetic*** Energy Operator                                                                                                                      	|
| $\sum_{I} \sum_{i} \frac{Z_I e^2}{\|R_I - r_i\|}$      	| ***Potential*** Energy Operator arising from the interactions between the electrons and nuclei. It is the sum of all electron-nucleus Coulomb interactions. 	|
| $\sum_i \sum_{j > i} \frac{e^2}{\|r_i - r_j\|}$         	| ***Potential*** Energy Operator arising from electron-electron Coulomb repulsions                                                                           	|
| $\sum_I \sum_{J > I} \frac{Z_I Z_J e^2}{\|R_I - R_J\|}$ 	| ***Potential*** Energy Operator arising from nuclear-nuclear repulsion energy operator, the sum of all nucleus-nucleus Coulomb repulsions.                  	|

#### In the **Born-Oppenheimer** approximation, we assume that the motions of the nuclie are uncoupled from the motions of the electrons, because the motions of the nuclei and those of the electrons are far different. To simplify, you can think that the position of nuclei are fixed(It is actually false because of the *uncertainty principle*, but it is just for the simplification). In the **Born-Oppenheimer** approximation, you solve for only the electronic equation with the fixed position of the nuclei. So, what we need to consider is just the motions of electrons. Let's define the hamiltonian related to the motions of electrons, which is only the deduction of only-nuclei-related-terms from the **Molecular Hamiltonian**: $$ \hat{H}_{el} = -\sum_{i} \frac{\nabla_{r_i}^{2}}{m_e} - \sum_{I} \sum_{i} \frac{Z_I}{|R_I - r_i|} + \sum_{i} \sum_{j > i} \frac{e^2}{|r_i - r_j|}$$

## III. Fock space

#### The electronic wave function $|\Psi\rangle_{elec}$ includes the spatial position for each electron ($r_P = \left\{x_p, y_p, z_p\right\}$) conditioned on the nuclear positions ($R_A = \left\{X_A, Y_A, Z_A\right\}$) and the spin directional coordinates for each electron ($s_p = \left\{s_{x_p}, s_{y_p}, s_{z_p}\right\}$). The electronic wave function must be antisymmetric with respect to the exchange of pair particle labels becuase these particles are fermions.

#### The **Fock space** is the Hilbert space in which the *Slater determinant wave functions* , which is antisymmetric with respect to the eschange of two electrons or with respect to the exchange of two spin orbitals, belong. By definition, a Fock space is the sum of a set of Hilbert spaces representing at least 3 important configurations:

- The zero-particle state is interpreted as the **absence** of an electron in an orbital: $|0\rangle$
- One particle states are interpreted as: $|r_p, s_p\rangle$ or $|1\rangle$
- Two particles states are interpreted as: $|r_i, s_i, r_j, s_j \rangle = \frac{1}{\sqrt{2}} \left(|r_j, s_j\rangle \otimes |r_i, s_i \rangle - v|r_i, s_i \rangle \otimes |r_j, s_j \rangle \right)$ or $|11\rangle$

#### Yes, this is only a mathematical construction and does not represent a physical reality or a chemical actuality. It can be seem useless, but the Fock space suggests the way to map the electron space of a molecule and the qubit space; but it is not a necessasry mapping. There are more computationally advantageous mapping methods and we will discuss about them later.

## IV. Fermionic Creation and Annihilation Operators

#### We define a set of fermionic annihilation operators, $\left\{\hat{a}_i \right\}_{i = 0}^{N - 1}$ and creation operators, $\left\{\hat{a}_{i}^{\dagger} \right\}_{j = 0}^{N - 1}$, following anti-commutation relations: $$ \left\{\hat{a}_i, \hat{a}_{j}^{\dagger} \right\} = \hat{a}_{i}^{\dagger}\hat{a}_{j} + \hat{a}_{j}\hat{a}_{i}^{\dagger} = \delta_{ij} = \begin{cases} 0, i \neq j & \\ 1, i = j & \end{cases} \\ \left\{\hat{a}_{i}^{\dagger}, \hat{a}_{j}^{\dagger} \right\} = \left\{\hat{a}_i, \hat{a}_{j} \right\} = 0$$ where $\delta_{ij}$ is the Dirac delta function.

#### A **fermionic operator** is a linear combination of products of **creation** and **annihilation operators**.

### IV.I Fermion creation operator

#### The fermionic creation operator $\hat{a}_{i}^{\dagger}$ raises by one unit number of particles sitting in the i<sup>th</sup> fermionic orbital: $$ \hat{a}_{i}^{\dagger} |\ldots m_i \ldots \rangle = \left(1 - m_i\right) (-1)^{\sum_{j < i} m_j} |\ldots (m + 1)_i \ldots \rangle $$ where:

- $m_i, (m+1)_i$ are the number of particles sitting in the i<sup>th<\sup> fermionic orbital
- $(1 - m_i)$ is a pre-factor that annihilates the state if we had an electron in the $i^{th}$ fermionic orbital, that is, if $m_i = 1$
- The phase factor $(-1)^{\sum_{j < i} m_j}$ keeps the anti-symmetric properties of the whole superposition of states.

### IV.II Fermion annhilation operator

#### The fermionic annhilation operator $\hat{a}_{i}^{\dagger}$ ,on the other hand, lowers by one unit number of particles sitting in the i<sup>th</sup> fermionic orbital: $$ \hat{a}_{i} |\ldots m_i \ldots \rangle = m_i (-1)^{\sum_{j < i} m_j} |\ldots (m - 1)_i \ldots \rangle $$ where:

- $m_i, (m-1)_i$ are the number of particles sitting in the $i^{th}$ fermionic orbital
- $m_i$ is a pre-factor that annihilates the state in the Slater determinant if there is no electron in the $i^{th}$ fermionic orbital, that is, if $m_i = 0$
- The phase factor $(-1)^{\sum_{j < i} m_j}$ keeps the anti-symmetric properties of the whole superposition of states.

## V. Molecular Hamiltonian in the Hatree-Fock orbitals basis

#### To map the original **electronic structure Hamiltonian** into the corresponding **qubit Hamiltonian**, we need to work in the second quantization formalism of quantum mechanics, which we have already introduced in the previous section, *Born-Oppenheimer approximation*

#### The **Hatree-Fock (HF)** method approximates an N-body problem into N one-body problems where each electron evolves in the **mean field** of other electrons.

#### Now, we can rewrite the electronic molecular Hamiltonian ($\hat{H}_{elec}$) as a linear combination of products of creation and annihilation operators: $$ \hat{H}_{elec} = \sum_{i, j} h_{pq} \hat{a}_{p}^{\dagger} \hat{a}_{q} + \frac{1}{2} \sum_{pqrs} h_{pqrs} \hat{a}_{p}^{\dagger} \hat{a}_{q}^{\dagger} \hat{a}_{r} \hat{a}_{s}$$

#### where $\hat{a}_{p}$ removes an electron from spin-orbital $p$, and $\hat{a}_{q}^{\dagger}$ creates an electron in spin-orbital $q$ The operation $\hat{a}_{p}^{\dagger} \hat{a}_{q}$ is the **excitation operator** which excites an electron from the **occupied spin-orbital** into the **unoccupied orbital**. These mappings are summarized below:

| **Molecular Hamiltonian in the Born-Oppenheimer approximation** | **Linear combination of products of creation and annihilation operators**                                                          | **Description**                                                             |
|-----------------------------------------------------------------|------------------------------------------------------------------------------------------------------------------------------------|-----------------------------------------------------------------------------|
| $\sum_{i} \frac{\nabla_{r_i}^{2}}{m_e}$                         | $\frac{1}{m_e} \sum_{pq} \langle p \mid \nabla_{p}^{2} \mid j \rangle \hat{a}_{p}^{\dagger} \hat{a}_{q}$                           | Electronic kinetic energy operator                                          |
| $\sum_{I} \sum_{i} \frac{Z_I}{\mid R_I - r_i \mid}$             | $\sum_{pq} \langle p \mid \frac{Z_A}{r_{pq}} \mid q \rangle \hat{a}_{p}^{\dagger} \hat{a}_{q}$                                     | Potential energy between the electrons and nuclei.                          |
| $\sum_{i} \sum_{j > i} \frac{e^2}{\mid r_i - r_j \mid}$         | $\sum_{pqrs} \langle pq \mid \frac{1}{r_{pq}} \mid rs \rangle \hat{a}_{p}^{\dagger} \hat{a}_{q}^{\dagger} \hat{a}_{r} \hat{a}_{s}$ | Potential energy operator arising from electron-electron Coulomb repulsions |

#### So, we can define $h_{pq}$ and $h_{pqrs}$

#### $$ h_{pq} = \langle p \mid \hat{h} \mid q \rangle = \int \phi_{p}^{*} (r) \left(-\frac{1}{2} \nabla^2 - \sum_{I} \frac{Z_I}{\mid R_I - r \mid}\right) \phi_q (r) dr $$

#### $$ h_{pqrs} = \langle pq \mid \hat{h} \mid rs \rangle = \int \frac{\phi_{p}^{\ast}(r_1) \phi_{q}^{\ast}(r_2) \phi_r(r_2) \phi_s(r_1)}{\mid r_1 - r_2 \mid} dr_1 dr_2 $$